In [1]:
import numpy as np
import pandas as pd 

import glob
import os


from scipy import stats
from scipy import signal

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy import signal
from scipy.fft import fft

import warnings
warnings.filterwarnings('ignore')


# importing all the necessary modules

import pandas as pd
import numpy as np
import random
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
# import lightgbm as lgb

from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.cluster import FeatureAgglomeration

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle

import warnings
warnings.filterwarnings('ignore')

random.seed(1729)

In [2]:
train = pd.read_csv('./Cleaned_Train.csv')
train.head()

,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [3]:
len(train['eeg_id'].unique()), len(train['spec_id'].unique())

(17089, 11138)

In [4]:
train.drop(columns = ['min','max','seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote', 'target'], axis = 1, inplace = True)

In [5]:
EEG_PATH = '../hms-harmful-brain-activity-classification/train_eegs'

# Create a DataFrame with the paths to all .parquet files under the base path
EEG_df = pd.DataFrame({'path': glob.glob(EEG_PATH + '**/*.parquet')})
# Extract test_type from the directory name
EEG_df['test_type'] = EEG_df['path'].str.split('/').str.get(-2).str.split('_').str.get(-1)
# Extract id from the file name
EEG_df['id'] = EEG_df['path'].str.split('/').str.get(-1).str.split('.').str.get(0)

# Read a specific Parquet file
df_eeg = pd.read_parquet(EEG_PATH + '/568657.parquet')
# Display the first few rows
df_eeg.head()

,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,-75.360001,86.379997,65.470001,62.580002,170.350006,92.370003,39.68,89.870003,18.260000,-28.440001,-15.190000,16.93,13.81,-42.160000,78.099998,-143.649994,121.239998,-1.470000,72.550003,-3090.090088
1,-107.739998,53.139999,32.590000,21.950001,140.500000,59.070000,4.26,53.310001,-18.490000,-63.270000,-52.110001,-13.23,-16.42,-80.430000,64.449997,-168.500000,88.400002,-28.520000,40.250000,5560.439941
2,-103.050003,46.330002,10.750000,9.150000,137.820007,50.950001,-3.50,45.070000,-20.879999,-66.629997,-69.750000,-9.20,-21.40,-93.089996,5.120000,-176.089996,51.689999,-38.639999,31.820000,-4161.450195
3,-75.099998,77.870003,40.150002,31.790001,166.630005,76.769997,17.99,67.650002,11.140000,-32.950001,-38.509998,14.77,9.79,-66.480003,48.220001,-147.990005,86.629997,-18.639999,62.509998,31769.970703
4,-97.919998,58.209999,27.330000,5.340000,144.080002,47.200001,-10.43,40.250000,-12.350000,-52.500000,-54.919998,-10.07,-8.04,-90.820000,67.269997,-164.710007,80.680000,-36.520000,41.830002,6879.720215


In [6]:
SPECTROGRAM_PATH = '../hms-harmful-brain-activity-classification/train_spectrograms'

# Create a DataFrame with the paths to all .parquet files under the base path
SPEC_df = pd.DataFrame({'path': glob.glob(SPECTROGRAM_PATH + '**/*.parquet')})
# Extract test_type from the directory name
SPEC_df['test_type'] = SPEC_df['path'].str.split('/').str.get(-2).str.split('_').str.get(-1)
# Extract id from the file name
SPEC_df['id'] = SPEC_df['path'].str.split('/').str.get(-1).str.split('.').str.get(0)

# Read a specific Parquet file
df_spectrogram = pd.read_parquet(SPECTROGRAM_PATH + '/978166025.parquet')
# Display the first few rows
df_spectrogram.head()

,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,LL_2.15,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def PCA_FeatureExtraction(spectrogram_df):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(spectrogram_df)
    pca = PCA(n_components = 1)
    X_pca = pca.fit_transform(X_scaled)

    # Define EEG frequency bands
    eeg_bands = {'Delta': (0.5, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 45)}
    fs = 200  # Sample rate

    # Function to filter and extract features
    def extract_features(segment):
        features = []
        for band in eeg_bands:
            low, high = eeg_bands[band]
            sos = signal.butter(3, [low, high], btype='bandpass', fs=fs, output='sos')
            filtered = signal.sosfilt(sos, segment)
            features.extend([np.mean(filtered), np.std(filtered), np.max(filtered), np.min(filtered)])
        return features

    # Apply feature extraction to each PCA component
    features_list = []
    for component in X_pca.T:  # Transpose to iterate over components
        component_features = extract_features(component)
        features_list.append(component_features)

    # Define column names for the DataFrame
    column_names = []
    for band in eeg_bands:
        for stat in ['mean', 'std', 'max', 'min']:
            column_names.append(f"{band}_{stat}")
    
    # Create the DataFrame from the features list
    features_df = pd.DataFrame(features_list, columns=column_names)

    return features_df


In [8]:
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy import signal
import numpy as np
import pandas as pd

def dbscan_pca_feature_extraction(df, eps=0.5, min_samples=5, pca_n_components=1):
    """
    Apply DBSCAN clustering, followed by PCA and feature extraction on the dataset.

    Parameters:
    df (pd.DataFrame): The input dataframe with the data.
    eps (float): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    min_samples (int): The number of samples in a neighborhood for a point to be considered as a core point.
    pca_n_components (int): Number of components for PCA.

    Returns:
    pd.DataFrame: A dataframe containing the PCA features and DBSCAN cluster labels.
    """
    
    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df)
    
    # Apply DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(X_scaled)
    
    # Check the number of clusters formed
    n_clusters_ = len(set(clusters)) - (1 if -1 in clusters else 0)
    
    # Apply PCA
    pca = PCA(n_components=pca_n_components)
    X_pca = pca.fit_transform(X_scaled)
    
    # Feature extraction from PCA components
    features = {
        'PCA_mean': np.mean(X_pca, axis=0),
        'PCA_std': np.std(X_pca, axis=0),
        'PCA_max': np.max(X_pca, axis=0),
        'PCA_min': np.min(X_pca, axis=0)
    }
    
    # Create a DataFrame with the features
    features_df = pd.DataFrame(features)
    
    # Add DBSCAN cluster labels to the features DataFrame
    features_df['DBSCAN_Cluster'] = clusters[0]  # Assuming the use of the first DBSCAN label for demonstration
    
    return features_df


In [26]:
# Function for Time Domain Features: Statistical Measures and Hjorth Parameters
def extract_time_domain_features(eeg_data):
    """
    Extracts time domain features from EEG data.
    
    :param eeg_data: A 2D numpy array where rows are samples and columns are EEG channels.
    :return: A dictionary with statistical measures and Hjorth parameters for each channel.
    """
    features = {}

    # Statistical measures
    features['mean'] = np.mean(eeg_data, axis=0)
    features['std_dev'] = np.std(eeg_data, axis=0)
    features['median'] = np.median(eeg_data, axis=0)

    # Hjorth Parameters
    diff_eeg_data = np.diff(eeg_data, axis=0)
    diff_diff_eeg_data = np.diff(diff_eeg_data, axis=0)

    activity = np.var(eeg_data, axis=0)
    mobility = np.sqrt(np.var(diff_eeg_data, axis=0) / activity)
    complexity = np.sqrt(np.var(diff_diff_eeg_data, axis=0) / np.var(diff_eeg_data, axis=0)) / mobility

    features['activity'] = activity
    features['mobility'] = mobility
    features['complexity'] = complexity

    df = pd.DataFrame(features)
    return df

df_eeg = pd.read_csv('./df_eeg.csv')
# Example usage of the function
time_domain_features = extract_time_domain_features(df_eeg)

# Display the keys of the extracted features for verification
time_domain_features.head()


,mean,std_dev,median,activity,mobility,complexity
Unnamed: 0,4999.500000,2886.751332,4999.500,8.333333e+06,0.000000,NaN
Fp1,-34.652174,50.661903,-35.575,2.566628e+03,1.402434,1.151075
F3,-10.938655,54.225023,-11.450,2.940353e+03,1.480064,1.091452
C3,-13.130097,58.494030,-12.970,3.421551e+03,1.508955,1.071061
P3,-11.532683,52.667059,-11.840,2.773819e+03,1.496873,1.079636


In [24]:
type(time_domain_features)

dict

In [28]:
# Re-importing necessary libraries as the code execution state was reset
import pandas as pd
from scipy.signal import butter, lfilter

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Applies a bandpass filter to the data.
    
    :param data: 2D array of EEG data (samples x channels)
    :param lowcut: Low cutoff frequency in Hz
    :param highcut: High cutoff frequency in Hz
    :param fs: Sampling frequency in Hz
    :param order: Order of the filter
    :return: Filtered data
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

# Load the EEG data
df_eeg = pd.read_csv('./df_eeg.csv')

# Assuming the EEG data starts from the second column (first column is index or time)
eeg_data = df_eeg.iloc[:, 1:].values

# Apply bandpass filter (0.5-45 Hz is a common range for EEG)
fs = 128  # Replace with the actual sampling frequency of your EEG data
filtered_eeg = bandpass_filter(eeg_data, lowcut=0.5, highcut=45, fs=fs, order=5)

# Creating a DataFrame of the filtered data for further analysis
filtered_eeg_df = pd.DataFrame(filtered_eeg, columns=df_eeg.columns[1:])
filtered_eeg_df.head()



,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,-20.788030,-17.524005,-15.605261,-9.668349,-19.586115,-17.237274,-10.473554,-9.964900,-6.333623,-8.268078,-8.497856,-17.425809,-14.613484,-18.158349,-11.573345,-14.874685,-11.679397,1.612373,13.058065,275.916177
1,-77.581013,-65.992768,-57.472767,-35.919033,-71.632673,-67.597319,-38.423144,-36.140181,-33.613653,-36.501927,-33.305012,-64.440830,-54.853115,-66.718683,-40.875568,-55.069075,-42.338537,43.245481,83.490446,1475.183220
2,-81.532840,-65.084831,-49.185790,-24.901472,-64.730118,-73.624089,-24.250180,-27.409575,-53.227150,-42.189748,-26.712184,-64.812609,-48.645534,-60.827955,-25.733099,-48.894414,-28.984339,102.140064,120.312123,1707.697205
3,0.433680,17.129182,32.969133,41.738092,27.548360,7.624727,51.185030,29.917782,-8.889437,15.779059,35.863967,5.683272,31.317275,25.069626,45.615859,27.475820,44.726047,4.254454,-61.221451,-1652.358900
4,-8.762428,5.994754,13.601831,25.966009,6.176716,6.053979,34.307498,14.676616,21.413905,24.849147,28.765177,-9.363364,17.117301,5.580843,22.155782,11.968433,23.104259,-107.576333,-133.416599,-2328.798896


In [29]:
ok = pd.read_csv('./df_eeg.csv')
ok.head()

,Unnamed: 0,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,...,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,0,-105.85,-89.23,-79.46,-49.23,-99.73,-87.77,-53.33,-50.74,-32.25,...,-43.27,-88.73,-74.41,-92.46,-58.93,-75.74,-59.47,8.21,66.49,1404.93
1,1,-85.47,-75.07,-60.26,-38.92,-73.08,-87.51,-39.68,-35.63,-76.84,...,-43.04,-68.63,-61.69,-69.32,-35.79,-58.90,-41.66,196.19,230.67,3402.67
2,2,8.84,34.85,56.43,67.97,48.10,25.35,80.25,48.06,6.72,...,61.00,16.58,55.06,45.02,70.53,47.82,72.03,-67.18,-171.31,-3565.80
3,3,-56.32,-37.28,-28.10,-2.82,-43.43,-35.05,3.91,-12.66,8.65,...,4.18,-51.90,-21.89,-41.33,-11.58,-27.04,-11.73,-91.00,-81.19,-1280.93
4,4,-110.14,-104.52,-96.88,-70.25,-111.66,-114.43,-71.83,-61.92,-76.15,...,-67.48,-99.03,-93.61,-104.41,-70.07,-89.25,-77.26,155.73,264.85,4325.37


In [31]:
import nolds
from fractal import HiguchiFractalDimension

def extract_nonlinear_features(eeg_data, emb_dim=2, tolerance=0.1):
    """
    Extracts nonlinear dynamics and complexity features from EEG data.
    
    :param eeg_data: A 2D numpy array where rows are samples and columns are EEG channels.
    :param emb_dim: Embedding dimension for entropy calculations.
    :param tolerance: Tolerance for entropy calculations.
    :return: DataFrame with nonlinear features for each channel.
    """
    features = {
        'approximate_entropy': [],
        'sample_entropy': [],
        'lyapunov_exponent': [],
        'fractal_dimension': []
    }

    for channel in eeg_data.T:
        # Approximate Entropy
        features['approximate_entropy'].append(nolds.app_entropy(channel, emb_dim=emb_dim, tolerance=tolerance))

        # Sample Entropy
        try:
            sampen = nolds.sampen(channel, emb_dim=emb_dim, tolerance=tolerance)
        except ValueError:
            sampen = float('nan')  # Handling cases where SampEn is undefined
        features['sample_entropy'].append(sampen)

        # Lyapunov Exponent
        try:
            lyap = nolds.lyap_r(channel, emb_dim=emb_dim)
        except ValueError:
            lyap = float('nan')  # Handling cases where Lyapunov exponent is undefined
        features['lyapunov_exponent'].append(lyap)

        # Fractal Dimension
        hfd = HiguchiFractalDimension(channel)
        features['fractal_dimension'].append(hfd)

    return pd.DataFrame(features)

# Applying the function to the filtered EEG data
nonlinear_features_df = extract_nonlinear_features(filtered_eeg)

nonlinear_features_df.head()



pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


ImportError: cannot import name 'HiguchiFractalDimension' from 'fractal' (/Users/kushalbanda/miniconda3/envs/Image_Captioning/lib/python3.11/site-packages/fractal/__init__.py)

In [21]:
time_domain_features.

AttributeError: 'dict' object has no attribute 'head'

In [9]:
All_Spec_df = []
for i in range(len(train)):
    df_spectrogram = pd.read_parquet(SPECTROGRAM_PATH + '/' + str(train['spec_id'][i]) + '.parquet')
    print(train['spec_id'][i])
    df_spectrogram.fillna(df_spectrogram.mean(), inplace=True)
    ok = dbscan_pca_feature_extraction(df_spectrogram)
    ok.insert(0, 'spec_id', train['spec_id'][i])

    All_Spec_df.append(ok)

    
# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(All_Spec_df, ignore_index=True)

# Now 'combined_df' contains all the data from the individual DataFrames
combined_df.head()

789577333
1552638400
14960202
618728447
52296320
2036345030
128369999
320962633
2074135650
1232582129
978166025
673742515
1211648246
223960986
2091405434
1331405712
1343094925
1459125071
1313185981
1902315832
1299007658
1683476984
1714949860
731432012
1069642612
182482796
619167662
944873540
1214465957
1497099492
2079726796
1966397930
1462669389
2593634
528989655
918152909
581115791
621122498
437709580
498849051
938288425
222655915
1762994934
1200923908
1421207755
203428911
1236865192
1514971337
184558222
1213593366
1505926989
1453123834
1125281567
178497203
327428082
1831189084
1052278850
1080089899
268885199
1022479026
125030030
825185801
357740047
300894776
130237761
1080327129
1098390103
1154290032
1289323695
1740925790
1965145273
1748356357
685420503
122348896
1010184710
1326122787
149028695
1456975380
874919861
1067342787
1841034439
1686467936
1538429652
1343094925
1857573314
1734113021
603308853
1211648246
1414053751
784433556
115188692
1848099717
1720444574
209992266
625758821


,spec_id,PCA_mean,PCA_std,PCA_max,PCA_min,DBSCAN_Cluster
0,789577333,1.291896e-15,19.261924,240.598954,-4.704242,-1
1,1552638400,0.000000e+00,13.660815,57.496412,-16.031266,-1
2,14960202,0.000000e+00,11.647719,110.324860,-10.831656,-1
3,618728447,2.870880e-16,12.895300,42.057751,-26.740369,-1
4,52296320,0.000000e+00,12.058408,95.127388,-11.011810,-1


In [10]:
combined_df.to_csv('DBSCAN_all_Spectrogram_df_1.csv', index = False)

In [11]:
combined_df

,spec_id,PCA_mean,PCA_std,PCA_max,PCA_min,DBSCAN_Cluster
0,789577333,1.291896e-15,19.261924,240.598954,-4.704242,-1
1,1552638400,0.000000e+00,13.660815,57.496412,-16.031266,-1
2,14960202,0.000000e+00,11.647719,110.324860,-10.831656,-1
3,618728447,2.870880e-16,12.895300,42.057751,-26.740369,-1
4,52296320,0.000000e+00,12.058408,95.127388,-11.011810,-1
...,...,...,...,...,...,...
17084,1188113564,-4.736952e-16,6.816192,33.903127,-12.300068,-1
17085,1549502620,0.000000e+00,8.751641,24.735109,-15.364318,-1
17086,2105480289,-1.326346e-15,19.868465,233.434097,-28.717455,0
17087,657299228,9.288140e-16,15.451703,264.941704,-3.311015,0


In [12]:
combined_df.head()

,spec_id,PCA_mean,PCA_std,PCA_max,PCA_min,DBSCAN_Cluster
0,789577333,1.291896e-15,19.261924,240.598954,-4.704242,-1
1,1552638400,0.000000e+00,13.660815,57.496412,-16.031266,-1
2,14960202,0.000000e+00,11.647719,110.324860,-10.831656,-1
3,618728447,2.870880e-16,12.895300,42.057751,-26.740369,-1
4,52296320,0.000000e+00,12.058408,95.127388,-11.011810,-1


In [13]:
combined_df.shape

(17089, 6)

In [14]:
combined_df.head()

,spec_id,PCA_mean,PCA_std,PCA_max,PCA_min,DBSCAN_Cluster
0,789577333,1.291896e-15,19.261924,240.598954,-4.704242,-1
1,1552638400,0.000000e+00,13.660815,57.496412,-16.031266,-1
2,14960202,0.000000e+00,11.647719,110.324860,-10.831656,-1
3,618728447,2.870880e-16,12.895300,42.057751,-26.740369,-1
4,52296320,0.000000e+00,12.058408,95.127388,-11.011810,-1
